In [0]:
---
title: "In 2014, Did Employers in the US Tech Industry Recognize the Importance of Mental Health?"
author: "Joseph Lisle"
date: "June 4, 2017"
output:
 html_document:
  theme: lumen
  highlight: tango
  number_sections: yes
  toc: yes
  code_folding: hide
---

# Introduction, Loading, and Cleaning {.tabset}
<b>Goal</b>  
 
A crucial aspect of a healthy and productive workplace is management's understanding of the importance of mental health, especially in fast-paced or high-growth sectors of the economy. Given the tech industry's rapid growth over the past few decades, I believe it would be valuable to examine the industry's employee access to mental health resources and their understanding of these resources.
 
I aim to answer the question: In 2014, did employers in the US tech industry recognize the importance of mental health?
 
<b>Benefits of Survey Data</b>
 
As this dataset includes thoughts from the workers' perspectives, it provides a closer look into gauging the need for mental health resources and whether resources currently offered are effective. To fully answer the question I propose, feedback from the employers would also be needed to compare the resources that are perceived to be available and which resources are actually available. With the limited data, however, I am still able to at least understand which resources employees perceive to be available. Additionally, the data provide insight as to whether an employee perceives that an employer recognizes the importance of mental health.
 
<b>Presentation Contents</b>
 
* Accordingly, I break down my analysis into three areas, which align closely with the variables provided in the dataset: 
    (1) Whether there is a need for mental health resources in the workplace, 
    (2) whether the employer is perceived to recognize the importance of mental health, and 
    (3) whether the employer offers resources to employees for mental health issues. 
 
<a href "https://www.kaggle.com/osmi/mental-health-in-tech-survey">The dataset can be found here.</a>
 
Comments are very welcome!
 
## Loading Packages
Look at code for details.
```{r Loading_Packages, warning = FALSE, message=FALSE}
# Loading packages
library(tidyverse)
library(plotly)
library(tidytext)
library(knitr)
library(ggplot2)
```
 
## Loading the Data and First Peek
Look at code for details.
 
Here are the `str` and `summary`.
```{r First_Peek}
# Loading data
Data <- read.csv("../input/survey.csv")
 
# Taking first peek
str(Data)
summary(Data)
```
 
## Cleaning Age and Timestamp
 
For the most part, it appears that the variables are clean. That being said, the gender and age variables need to be cleaned and the timestamp should be separated to to access the dates within.

In the code, I clean the age variable and separate the timestamp. The spread of respondents genders' is below.
 
```{r Cleaning_Age_Timestamp}
# Cleaning age variable
Data$Age <-
  ifelse(Data$Age < 12, "", # I think we can all safely say that anyone younger than 12 does not "work" in the tech industry
  ifelse(Data$Age > 150, "", Data$Age)) %>% # No one is this old, so this will probably solve the higher numbers problem
  as.numeric()
 
# Separating timestamp
Data$Timestamp <- as.POSIXct(strptime(Data$Timestamp, format = "%Y-%m-%d %H:%M:%S"))
 
# Looking into genders to understand how to clean
table(Data$Gender)
```
 
## Cleaning Gender
 
* Notes before I "clean" the gender variable:
    + I love the representation of non-binary and inter-dichotomy genders in this dataset!
    + Although there are quite a few participants who specify whether they are trans, not everyone was asked to make such a distinction. Thus, I include them in the gender they identify as (i.e. "Trans-female" being included in "Female").
    + Although it is important to not marginalize someone because of their gender, there are so few people identifying as non-male/non-female that I place them all in one category for ease of analysis.
 
```{r Cleaning_Gender}
# Setting up the entries into gender "categories" for ease of analysis
GenMale <- c("cis male", "Cis Male", "Cis Man", "m", "M", "Mail", "maile", "Make", "Mal", "male", "Male", "Male ", "Male (CIS)", "Malr", "Man", "msle")
GenFemale <- c("cis-female/femme", "Cis Female", "f", "F", "femail", "Femake", "female", "Female", "Female ", "Female (cis)", "Female (trans)", "Trans-female", "Trans woman", "woman", "Woman")
 
# Assigning the entries according to "categories"
Data$GenderClean <-
  ifelse((Data$Gender %in% GenMale), "Male", # Assigning "Male" to those who entered a string contained in GenMale
  ifelse((Data$Gender %in% GenFemale), "Female", "Non-M/F")) %>% # Assigning "Female" to those who entered a string contained in GenFemale
  as.factor()
 
# Observing cleaned table
table(Data$GenderClean)
```
 
Now that the cleaning is completed, let's get the data that is needed for analysis to answer the primary question.
 
## Filtering Data
Here's the final `summary`!
```{r Filter_Data}
# Creating new dataset with filtered rows
TechData <- Data %>%
  filter(tech_company == "Yes" & # Making sure dataset includes only those in a tech company
         format(Timestamp, "%Y") == 2014 & # Making sure dataset includes only responses from 2014
         Country == "United States") # Making sure dataset includes only responses from US
 
summary(TechData)
```
 
# Analysis
 
* As noted in the introduction, I break down my analysis into three areas: 
    (1) Whether there is a need for mental health resources in the workplace, 
    (2) whether the employer is perceived to recognize the importance of mental health, and 
    (3) whether the employer offers resources to employees for mental health issues.
 
## Need for Mental Health Resources {.tabset}
<b>Mental Health Condition Prevalence</b>
 
A large portion of the United States population faces issues with mental health. <a href="https://www.nimh.nih.gov/health/statistics/prevalence/any-mental-illness-ami-among-us-adults.shtml">According to the National Institute of Mental Health (NIMH), about 17.9% of the US population experienced a mental illness from 2014-2015.</a> Naturally, any survey taken will have some bias, but the NIMH notes that about 1/3 of the sample did not complete the survey and that those with mental illness may disproportionately fall into this category. 
 
It is important to note that the NIMH survey only counted mental illnesses that are diagnosable or disorders are mentally-, behaviorally-, or emotionally-based. Thus, the self-reporting from the respondents may not reflect the actual prevalence of mental health issues. The consequences of this are discussed in the conclusion section.
 
<b>Indicators of Mental Health Prevalence in Dataset</b>
 
* There are three possible indicators of the prevalence of mental health issues:
    (1) Family History:  Do you have a family history of mental illness?
    (2) Treatment: Have you sought treatment for a mental health condition?
    (3) Work Interference: If you have a mental health condition, do you feel that it interferes with your work?
 
<b>Things to Keep in Mind</b>
 
Though asking about treatment can and does weed out self-diagnosis of mental illness, 'treatment' and 'mental illness' are not clearly defined. Additionally, 'mental health condition' and "mental illness" seemed to be used interchangeably throughout the survey. Consequently, by responding "yes" to seeking treatment for a mental health condition, respondents may mean that they have had therapy for a breakup or that they had pills described to them for a diagnosis of depression. 
 
This bias is incredibly important to keep in mind when looking at the results, especially considering that the answers to this question are the primary indicators of mental  for the analysis. Note that, in this analysis, I use 'mental illness' and 'mental health condition' interchangeably unless specified by the survey.
 
<b>Graphs</b>
 
Here's a look at these variables. Note that I include the questions asked to the respondents before the figures so that the responses are given further context.
 
### Family History
<h3><center>Do you have a family history of mental illness?</center></h3>
```{r Prevalence_Family_History}
# Setting up colors for clarity
YesNoMaybe <- c("No" = "#ed2828", "Yes" = "2bed28", "Maybe" = "#ede628", "Not sure" = "#ede628", "Don't know" = "#ede628")
 
# Pie Chart time!
TechData %>%
  count(family_history) %>% # Counting the people in "family_history" to put in chart
  plot_ly(
    labels = ~family_history,
    values = ~n,
    type = "pie", # Setting type of chart to pie
    textposition = 'inside', # Setting area for text
    textinfo = 'label+percent', # Setting labels
    hoverinfo = 'text', # Setting text on hover (see text variable on next line)
    text = ~paste(n, "Respondents"), # Setting text on hover
    marker = list(colors = YesNoMaybe)) %>% # Setting up colors for clarity
  layout(title = "Responses")
```

Based on the sample, a large percentage of people in the tech industry in the US have a family history of mental illness.
 
### Treatment
<h3><center>Have you sought treatment for a mental health condition?</center></h3>
```{r Prevalence_Treatment}
TechData %>%
  count(treatment) %>% # Counting the people in "treatment" to put in chart
  plot_ly(
    labels = ~treatment,
    values = ~n,
    type = "pie", # Setting type of chart to pie
    textposition = 'inside', # Setting area for text
    textinfo = 'label+percent', # Setting labels
    hoverinfo = 'text', # Setting text on hover (see text variable on next line)
    text = ~paste(n, "Respondents"), # Setting text on hover
    marker = list(colors = YesNoMaybe)) %>% # Setting up colors for clarity
  layout(title = "Responses")
```

Based on the responses (assuming it is a representative sample), it looks like a _much_ larger percentage of people in the tech industry experience mental health issues, indicated by their seeking of treatment.
 
### Work Interfere
<h3><center>If you have a mental health condition, do you feel that it interferes with your work?</center></h3>
```{r Prevalence_Work_Interfere}
# Setting color scheme for clarity
WICol <- c("Often" = "#2bed28", "Never" = "#ed2828", "Rarely" = "#ede628", "Sometimes" = "#eaa927", "No Answer" = "#bababa")
 
# Pie chart!
TechData %>%
  count(work_interfere) %>% # Counting the people in "work_interfere" to put in chart
  plot_ly(
    labels = ~work_interfere,
    values = ~n,
    type = "pie", # Setting type of chart to pie
    textposition = 'inside', # Setting area for text
    textinfo = 'label+percent', # Setting labels
    hoverinfo = 'text', # Setting text on hover (see text variable on next line)
    text = ~paste(n, "Respondents"), # Setting text on hover
    marker = list(colors = WICol)) %>% # Setting up colors for clarity
  layout(title = "Responses")
```

Nearly 50% of the sample indicated that they sometimes or often encounter interference with their work due to a mental health condition, whereas the other 50% did not answer, or indicated they never or rarely encounter such interference.
 
## Perceptions of Employer Recognition of the Importance of Mental Health {.tabset}
 
Now that we see that a large percentage (over half!) of the sample experience mental health issues (as indicated by their seeking of treatment), I am interested to see how the recognition of the importance of mental health is perceived. 
 
To better understand if the needs are being fulfilled, I break down the groups into two categories of people: those who have sought treatment for a mental health condition and those who have not.
 
### Mental Health Leave
<h3><center>How easy is it for you to take medical leave for a mental health condition?</center></h3>
```{r Perceptions_Leave, message=FALSE, warning=FALSE}
# Specifying the order of bars for the chart
LeaveBarOrder <- c("Very difficult", "Somewhat difficult", "Somewhat easy", "Very easy", "Don't know")
 
# Editing values for treatment variable for ease of communication in charts
TechData$treatment <-
  ifelse(TechData$treatment == "No", "Not Sought Treatment",
  ifelse(TechData$treatment == "Yes", "Sought Treatment", TechData$treatment)) %>%
  as.factor()
 
# Here's the chart!
TechData %>%
  count(leave, treatment) %>% # Counting people to add in chart
  plot_ly(
    x = ~leave,
    y = ~n,
    type = "bar", # Setting type of plot to bar
    hoverinfo = 'text', # Setting hover info (look at "text" on next line)
    text = ~paste(n, "Respondents"), # Setting hover info
    color = ~treatment) %>% # setting colors of bars
  layout(title = "", # Setting title
         xaxis = list(title = "Responses", # Setting up x-axis title
                      categoryorder = 'array',
                      categoryarray = LeaveBarOrder), # Setting up order of bars
         yaxis = list(title = "Number of Respondents"), # Setting up y-axis title
         legend = list(x = 0.1, y=0.9)) # Moving legend
```

* As I suspected, there are some differences in the perceptions of those who have sought treatment for mental  vs. those who have not:
    + Those who have sought treatment for mental  perceive it to be more difficult to take medical leave for their mental 
    + That being said, the numbers are fairly similar.
* Regardless, a large number of people do not know how easy it would be to take medical leave for mental .
 
### Mental Health Consequence
<h3><center>Do you think that discussing a mental health issue with your employer would have negative consequences?</center></h3>
```{r Perceptions_Mental_Health_Consequence, message=FALSE, warning=FALSE}
# Specifying the order of bars for the chart
YNMBarOrder <- c("No", "Maybe", "Yes")
 
# Here's the chart!
TechData %>%
  count(mental_health_consequence, treatment) %>% # Counting people by category to add in chart
  plot_ly(
    x = ~mental_health_consequence,
    y = ~n,
    type = "bar", # Setting type of plot to bar
    hoverinfo = 'text', # Setting hover info (look at "text" on next line)
    text = ~paste(n, "Respondents"), # Setting hover info
    color = ~treatment) %>% # setting colors of bars
  layout(title = "", # Setting title
         xaxis = list(title = "", # Setting up x-axis title
                      categoryorder = 'array',
                      categoryarray = YNMBarOrder), # Setting up order of bars
         yaxis = list(title = "Number of Respondents"),
         legend = list(x = 0.7, y = 0.95)) # Setting up y-axis title
```

It seems there are distinct differences between the groups here as well.
 
Those who have sought treatment perceive there to be negative consequences when discussing mental health issues with an employer more often that those who have not sought treatment.
 
### Discussion with Coworkers
<h3><center>Would you be willing to discuss a mental health issue with your coworkers?</center></h3>
```{r Perceptions_Coworkers, message=FALSE, warning=FALSE}
# Specifying the order of bars for the chart
YSMBarOrder <- c("No", "Some of them", "Yes")
 
# Here's the chart!
TechData %>%
  count(coworkers, treatment) %>% # Counting people by category to add in chart
  plot_ly(
    x = ~coworkers,
    y = ~n,
    type = "bar", # Setting type of plot to bar
    hoverinfo = 'text', # Setting hover info (look at "text" on next line)
    text = ~paste(n, "Respondents"), # Setting hover info
    color = ~treatment) %>% # setting colors of bars
  layout(title = "", # Setting title
         xaxis = list(title = "Responses", # Setting up x-axis title
                      categoryorder = 'array',
                      categoryarray = YSMBarOrder), # Setting up order of bars
         yaxis = list(title = "Number of Respondents"), # Setting up y-axis title
         legend = list(x = 0.7, y=0.9)) # Moving legend
```

Though willingness to discuss with coworkers does not directly relate to policies or resources that employers can provide, it provides insight into the culture of a company and whether employers foster a healthy environment wherein employees can freely speak.
 
Overall, it appears that most employees feel comfortable speaking to some or all coworkers. Additionally, it does not appear there are large differences between the groups, which probably indicates that workers perceive their environments to be the same regardless of whether they sought treatment for a mental health condition.
 
### Supervisor
<h3><center>Would you be willing to discuss a mental health issue with your direct supervisor(s)?</center></h3>
```{r Perceptions_Supervisor, message=FALSE, warning=FALSE}
TechData %>%
  count(supervisor, treatment) %>% # Counting people by category to add in chart
  plot_ly(
    x = ~supervisor,
    y = ~n,
    type = "bar", # Setting type of plot to bar
    hoverinfo = 'text', # Setting hover info (look at "text" on next line)
    text = ~paste(n, "Respondents"), # Setting hover info
    color = ~treatment) %>% # setting colors of bars
  layout(title = "", # Setting title
         xaxis = list(title = "Responses", # Setting up x-axis title
                      categoryorder = 'array',
                      categoryarray = YSMBarOrder), # Setting up order of bars
         yaxis = list(title = "Number of Respondents"), # Setting up y-axis title
         legend = list(x = 0.1, y=0.95)) # Moving legend
```

Again, like with the coworker variable, a majority of employees are willing to discuss mental health issues with at least some of their supervisors. Compared with the coworker variable, however, employees seem to be less willing to discuss mental health issues with supervisors.
 
That being said, there are large differences between those who have/have not sought treatment. Specifically, people who have sought treatment are only willing to speak to some or none of their supervisors more often than those who have not sought treatment.
 
### Observed Consequence
<h3><center>Have you heard of or observed negative consequences for coworkers with mental health conditions in your workplace?</center></h3>
```{r Perceptions_Observed_Consequence, message=FALSE, warning=FALSE}
# Specifying the order of bars for the chart
NYBarOrder <- c("No", "Yes")
 
# Here's the chart!
TechData %>%
  count(obs_consequence, treatment) %>% # Counting people by category to add in chart
  plot_ly(
    x = ~obs_consequence,
    y = ~n,
    type = "bar", # Setting type of plot to bar
    hoverinfo = 'text', # Setting hover info (look at "text" on next line)
    text = ~paste(n, "Respondents"), # Setting hover info
    color = ~treatment) %>% # setting colors of bars
  layout(title = "", # Setting title
         xaxis = list(title = "Responses", # Setting up x-axis title
                      categoryorder = 'array',
                      categoryarray = NYBarOrder), # Setting up order of bars
         yaxis = list(title = "Number of Respondents"), # Setting up y-axis title
         legend = list(x = 0.6, y=0.95)) # Moving legend
```

Overall, a vast majority of people have not seen negative consequences in their workplace for others with mental health issues. This is great! 
 
That being said, people who have sought treatment for mental health issues have observed more negative consequences.
 
## Actual Availability of Resources {.tabset}
 
Here is a look into whether resources are available to employees for mental health! Again, I break down into those who have sought treatment and those who have not.
 
### Benefits
<h3><center>Does your employer provide mental health benefits?</center></h3>
```{r Resources_Benefits, message=FALSE, warning=FALSE}
# Specifying the order of bars for the chart
NYDBarOrder <- c("No", "Yes", "Don't know")
 
# Here's the chart!
TechData %>%
  count(benefits, treatment) %>% # Counting people by category to add in chart
  plot_ly(
    x = ~benefits,
    y = ~n,
    type = "bar", # Setting type of plot to bar
    hoverinfo = 'text', # Setting hover info (look at "text" on next line)
    text = ~paste(n, "Respondents"), # Setting hover info
    color = ~treatment) %>% # setting colors of bars
  layout(title = "", # Setting title
         xaxis = list(title = "Responses", # Setting up x-axis title
                      categoryorder = 'array',
                      categoryarray = NYBarOrder), # Setting up order of bars
         yaxis = list(title = "Number of Respondents"), # Setting up y-axis title
         legend = list(x = 0.1, y=0.95)) # Moving legend
```

For the most part, it looks like a majority of respondents indicated that mental health benefits are provided by an employer. Unsurprisingly, those who have sought treatment know better than those who have not whether their employer does or does not provide these benefits.
 
### Care Options
<h3><center>Do you know the options for mental health care your employer provides?</center></h3>
```{r Resources_Care_Options, message=FALSE, warning=FALSE}
# Specifying the order of bars for the chart
NYNBarOrder <- c("No", "Yes", "Not sure")
 
# Here's the chart!
TechData %>%
  count(care_options, treatment) %>% # Counting people by category to add in chart
  plot_ly(
    x = ~care_options,
    y = ~n,
    type = "bar", # Setting type of plot to bar
    hoverinfo = 'text', # Setting hover info (look at "text" on next line)
    text = ~paste(n, "Respondents"), # Setting hover info
    color = ~treatment) %>% # setting colors of bars
  layout(title = "", # Setting title
         xaxis = list(title = "Responses", # Setting up x-axis title
                      categoryorder = 'array',
                      categoryarray = NYNBarOrder), # Setting up order of bars
         yaxis = list(title = "Number of Respondents"), # Setting up y-axis title
         legend = list(x = 0.1, y=0.95)) # Moving legend
```

Again, similar to the benefits variable, it is unsurprising that many more employees who have sought treatment know of the mental health care options available to them from their employer. 
 
That being said, a majority of people who have not sought treatment _do not know_ what care options are available to them from their employer. 
 
### Wellness Program
<h3><center>Has your employer ever discussed mental health as part of an employee wellness program?</center></h3>
```{r Resources_Wellness_Program, message=FALSE, warning=FALSE}
TechData %>%
  count(wellness_program, treatment) %>% # Counting people by category to add in chart
  plot_ly(
    x = ~wellness_program,
    y = ~n,
    type = "bar", # Setting type of plot to bar
    hoverinfo = 'text', # Setting hover info (look at "text" on next line)
    text = ~paste(n, "Respondents"), # Setting hover info
    color = ~treatment) %>% # setting colors of bars
  layout(title = "", # Setting title
         xaxis = list(title = "Responses", # Setting up x-axis title
                      categoryorder = 'array',
                      categoryarray = NYDBarOrder), # Setting up order of bars
         yaxis = list(title = "Number of Respondents"), # Setting up y-axis title
         legend = list(x = 0.7, y=0.95)) # Moving legend
```

The great majority of people report their employer not discussing mental health as a part of an employee wellness program. Unsurprisingly, a greater proportion of employees who have sought treatment answered either "yes" or "no", indicating that they are more aware of their employer communications regarding the subject. 
 
### Seek Help
<h3><center>Does your employer provide resources to learn more about mental health issues and how to seek help?</center></h3>
```{r Resources_Seek_Help, message=FALSE, warning=FALSE}
TechData %>%
  count(seek_help, treatment) %>% # Counting people by category to add in chart
  plot_ly(
    x = ~seek_help,
    y = ~n,
    type = "bar", # Setting type of plot to bar
    hoverinfo = 'text', # Setting hover info (look at "text" on next line)
    text = ~paste(n, "Respondents"), # Setting hover info
    color = ~treatment) %>% # setting colors of bars
  layout(title = "", # Setting title
         xaxis = list(title = "Responses", # Setting up x-axis title
                      categoryorder = 'array',
                      categoryarray = NYDBarOrder), # Setting up order of bars
         yaxis = list(title = "Number of Respondents"), # Setting up y-axis title
         legend = list(x = 0.7, y=0.95)) # Moving legend
```
</center>
A large percentage of employers do not provide resources for more information and seeking help, according to the responses. Alarmingly, a large percentage of people do not know whether this is the case.
 
As with the wellness program, a greater proportion of people who have sought treatment for mental health issues know whether an employer provides resources for seeking help or for more information when compared to those who have not sought treatment.
 
### Anonymity
<h3><center>Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources?</center></h3><center>
```{r Resources_Anonymity, message=FALSE, warning=FALSE}
TechData %>%
  count(anonymity, treatment) %>% # Counting people by category to add in chart
  plot_ly(
    x = ~anonymity,
    y = ~n,
    type = "bar", # Setting type of plot to bar
    hoverinfo = 'text', # Setting hover info (look at "text" on next line)
    text = ~paste(n, "Respondents"), # Setting hover info
    color = ~treatment) %>% # setting colors of bars
  layout(title = "", # Setting title
         xaxis = list(title = "Responses", # Setting up x-axis title
                      categoryorder = 'array',
                      categoryarray = NYDBarOrder), # Setting up order of bars
         yaxis = list(title = "Number of Respondents"), # Setting up y-axis title
         legend = list(x = 0.05, y=0.95)) # Moving legend
```

Alarmingly, a huge proportion of respondents do not know whether their anonymity would be protected it utilizing resources that their employer offers. 
 
That being said, a larger proportion of those who have sought treatment do know whether this is the case, perhaps indicating that they have utilized the resources their employer offers.
 
# Conclusions and Recommendations
 
* Before jumping to conclusions, it is important to understand some of the limitations of this data. These include, but are not limited to, the following:
    + Given that anyone can visit <a href="https://osmihelp.org/">Open Source Mental Illness</a>'s website and take the survey, there is probably a lot of bias in the representation of the members of the industry.
    + The purpose of the organization is to raise awareness of mental health in the tech and open source workplace. Though I applaud this goal (it truly is an amazing one!), the questions may be formatted as to bring about a specific conclusion aligning with their end-goal.
    + More will be discussed in the coming sections!
 
## Need for Resources
 
### Measuring Mental Health Prevalence {.tabset}
<b>Sought Treatment Variable</b>

Although the measure of mental illness is difficult in general, the closest measure in this dataset is the "sought treatment" variable, which indicates whether a respondent has sought treatment for a mental health condition. 

<b>Family History and Work Interference Variables</b>

In the section above, I also included the "family history" and "work interfere" variables, through which respondents indicate whether they have a family history of mental illness and how much their mental health condition interferes with their work, respectively. 

As we can see by the tables below, those with a family history of mental illness have higher rates of having sought treatment, and those who experience work interference due to a mental health condition are more likely to have sought treatment for a mental health condition.

Accordingly, these variables all roughly articulate the same thing - the prevalence of mental health issues in our sample.

#### Similarities: Treatment and Family History
<h3><center>Do you have a family history of mental illness?</center></h3>
```{r Similarities_Treatment_Family_History, warning=FALSE, message=FALSE}
TechData %>%
  count(family_history, treatment) %>% # Counting people by category to add in chart
  plot_ly(
    x = ~family_history,
    y = ~n,
    type = "bar", # Setting type of plot to bar
    hoverinfo = 'text', # Setting hover info (look at "text" on next line)
    text = ~paste(n, "Respondents"), # Setting hover info
    color = ~treatment) %>% # setting colors of bars
  layout(title = "", # Setting title
         xaxis = list(title = "Responses"), # Setting up x-axis title
         yaxis = list(title = "Number of Respondents")) # Setting up y-axis title
```


#### Similarities: Treatment and Work Interference
<h3><center>If you have a mental health condition, do you feel that it interferes with your work?</center></h3>
```{r Similarities_Treatment_Work_Interfere, warning=FALSE, message=FALSE}
TechData %>%
  count(work_interfere, treatment) %>% # Counting people by category to add in chart
  plot_ly(
    x = ~work_interfere,
    y = ~n,
    type = "bar", # Setting type of plot to bar
    hoverinfo = 'text', # Setting hover info (look at "text" on next line)
    text = ~paste(n, "Respondents"), # Setting hover info
    color = ~treatment) %>% # setting colors of bars
  layout(title = "", # Setting title
         xaxis = list(title = "Responses"), # Setting up x-axis title
         yaxis = list(title = "Number of Respondents")) # Setting up y-axis title
```


### Are Resources Needed?
<center><h4><i>Short answer: It's complicated!</i></h4></center>
 
<b>Possible Conclusions</b>

* If these variables indeed indicate the prevalence of mental illness in the tech industry, then the rates are incredibly different from the general population. The results from this sample suggest that either:
    (1) the rates of mental illness in the tech industry are much higher than outside the tech industry, 
    (2) the rates of mental illness in the tech industry are similar to outside the industry and the sample was incredibly biased in this respect, or 
    (3) a mixture of the two.
    
<b>Possible Consequences</b>

* If (1) is true, then the need for mental health resources and employer recognition is very high, at least compared with other industries.
* If (2) is true, we cannot draw any conclusions on prevalence and need for resources, and thus would need a more representative sample before making conclusions about people outside our sample.
* If (3) is true, more data would need to be collected, but it would appear that the prevalence rate of mental illness is still higher in the tech industry, indicating a need for resources.
 
<b>Final Notes on Necessity of Resources</b>

* Ultimately, it would be improper to draw any of these conclusions due to lack of data and context. 
* That being said, due to the number of people who responded, there are some valuable insights provided regarding the perception of recognition by employers and the actual availability of resources, especially for those with mental health conditions.
 
## Do Employers Recognize the Importance of Mental Health?
<center><h4><i>Short answer: Inconclusive</i></h4></center>

<b>Environment May be Open Among Coworkers...</b>

* But many fewer employees feel comfortable speaking to their supervisors about mental health. This may indicate that the work environment is open, to an extent, and that social structures are in place to allow coworkers to feel welcomed.

<b>Lack of Systems</b>

* However, it does not appear that there are systems in place for employees to discuss or take medical leave for mental health issues. 
* If these systems were in place, many more employees would know how easy it is to take medical leave for a mental health condition. 
* Furthermore, a majority of respondents would not feel that discussing their mental health issues might cause negative consequences and more would be willing to discuss their mental health issues with their supervisors.

<b>Conclusion</b>
* Ultimately, the variables in this dataset do not allow a firm conclusion to be made as to whether employers recognize the importance of mental health.
* But it appears that the tech industry may be on the right track, as many employees are willing to discuss their mental health issues with at least of their coworkers and supervisors. 
* The number of observed negative consequences for those with mental health conditions seems low, though a baseline would be helpful in determining whether the tech industry's rate is actually low.
 
## Do Employers Offer Benefits/Resources for Mental Health Issues?
<center><h4><i>Short answer: Maybe?</i></h4></center>

<b>There are Definitely _Some_ Resources</b>

* As indicated by the high number of affirmative responses for questions asking if employers offer mental health benefits or resources, it appears there are at least some resources available to employees.

<b>Communication About Resources Could Be Better</b>

* Across all the questions regarding availability and knowledge of benefits, many respondents indicated that they did not know what benefits or resources were available to them.
* Furthermore, over half of respondents answered that their employer has not discussed mental health as a part of an employee wellness program.